First, photos of sick dogs are preprocessed and replaced in their
original directories.
For each image:
    dog's face is detected,
    photo is rotated,
    photo is cropped to 100 x 100 px size.

In [1]:
from BayesInference import read_data, BayesInference
import numpy as np
# from CyfroVet.ML_Veterinray_Metrics_Func import img_preprocess
# input_data_paths = read_data('/Users/hwdowiak/Desktop/inzynierka/data/final_photos')
# for path, disease, age_category in input_data_paths:
#     img_preprocess(path+'/', '/Users/hwdowiak/Desktop/inzynierka/data/thrash')

2022-01-10 04:21:26.734415: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Then we're preparing a dictionary of all possibly occurring diseases.

In [2]:
input_data_paths = read_data('', only_dirs=False)
input_data_paths_heatmaps = read_data('', only_dirs=False)
all_diseases = set()
for dir in input_data_paths:
    for disease in dir[1]:
        all_diseases.add(disease)
enumerated_diseases = {disease: i for i, disease in enumerate(all_diseases)}


'input_data_paths' variable contains a list of image paths, diseases and age categories of dogs for each subdirectory.

In [3]:
from sklearn.model_selection import train_test_split
train_paths, test_paths = train_test_split(input_data_paths, random_state=42)
train_paths_heatmaps, test_paths_heatmaps = train_test_split(input_data_paths_heatmaps, random_state=42)

We can create Bayes inference instance, load all preprocessed pictures and check,
what will be a prediction for a new picture. It will be a photo of 2 year old, completely blind, mixed-shepherd.

In [4]:
bi = BayesInference(disease_categories=enumerated_diseases, age_categories={'junior': 0, 'adult': 1, 'senior': 2})
for path, disease, age_category in train_paths:
    bi.load_data_with_path(image_directory_path=path, age_on_images=age_category, diseases=disease, load_heatmap=False)

bi_heatmaps = BayesInference(disease_categories=enumerated_diseases, age_categories={'junior': 0, 'adult': 1, 'senior': 2})
for path, disease, age_category in train_paths_heatmaps:
    bi_heatmaps.load_data_with_path(image_directory_path=path, age_on_images=age_category, diseases=disease, load_heatmap=False)

Now we can experiment with hyperparams:
    - pixel threshold (how many % of identical pixels should two photos have to be considered identical)
    - amount of bits used to encode each pixel
    - We can also apply pooling to our images.

It is possible to declare size of slices into which, the images should be divided.
Second pooling parameter, is the function, used to reduce data from each slice.

In [5]:
from CyfroVet.bayes.utils import weighted_f_score
import numpy as np

for encoding_cats in [8, 16, 20]:
    for sim_threshold in [0.6, 0.7, 0.8, 0.9]:
        for size in [2, 5, 10, 20]:
            for func in [np.max, np.min, np.mean, np.median]:
                predictions = []
                prior_based_predicitons = []
                diseases = []
                for path, disease, age_category in test_paths:
                    result = bi.get_diseases_probabilities(new_image_path=path,
                                                           encoding_categories=encoding_cats,
                                                           similarity_threshold=sim_threshold,
                                                           pooling_slice_area=size,
                                                           pooling_function=func)
                    result_prior_based = bi.get_diseases_probabilities(new_image_path=path,
                                                           encoding_categories=encoding_cats,
                                                           similarity_threshold=sim_threshold,
                                                           pooling_slice_area=size,
                                                           pooling_function=func,
                                                           only_prior=True)
                    predictions.append(result)
                    prior_based_predicitons.append(result_prior_based)
                    diseases.append((*disease,))
                f1 = weighted_f_score(predictions, diseases)
                f1_prior_based = weighted_f_score(prior_based_predicitons, diseases)
                print(f'score={f1}, prior-based-score={f1_prior_based}, area_size={size}, function={func.__name__}, sim_threshold={sim_threshold}, encoding_categories={encoding_cats}')

score=0.7175337707804019, prior-based-score=0.7175337707804019, area_size=2, function=amax, sim_threshold=0.6, encoding_categories=2
score=0.7175337707804019, prior-based-score=0.7175337707804019, area_size=2, function=amin, sim_threshold=0.6, encoding_categories=2
score=0.7175337707804019, prior-based-score=0.7175337707804019, area_size=2, function=mean, sim_threshold=0.6, encoding_categories=2
score=0.7175337707804019, prior-based-score=0.7175337707804019, area_size=2, function=median, sim_threshold=0.6, encoding_categories=2
score=0.7175337707804019, prior-based-score=0.7175337707804019, area_size=5, function=amax, sim_threshold=0.6, encoding_categories=2
score=0.7175337707804019, prior-based-score=0.7175337707804019, area_size=5, function=amin, sim_threshold=0.6, encoding_categories=2
score=0.7175337707804019, prior-based-score=0.7175337707804019, area_size=5, function=mean, sim_threshold=0.6, encoding_categories=2
score=0.7175337707804019, prior-based-score=0.7175337707804019, are

In [6]:
for encoding_cats in [2, 4, 8, 16]:
    for sim_threshold in [0.6, 0.7, 0.8, 0.9]:
        for size in [2, 5, 10, 20]:
            for func in [np.max, np.min, np.mean, np.median]:
                predictions = []
                prior_based_predicitons = []
                diseases = []
                for path, disease, age_category in test_paths_heatmaps:
                    result = bi_heatmaps.get_diseases_probabilities(new_image_path=path,
                                                           encoding_categories=encoding_cats,
                                                           similarity_threshold=sim_threshold,
                                                           pooling_slice_area=size,
                                                           pooling_function=func)
                    result_prior_based = bi.get_diseases_probabilities(new_image_path=path,
                                                           encoding_categories=encoding_cats,
                                                           similarity_threshold=sim_threshold,
                                                           pooling_slice_area=size,
                                                           pooling_function=func,
                                                           only_prior=True)
                    predictions.append(result)
                    prior_based_predicitons.append(result_prior_based)
                    diseases.append((*disease,))
                f1 = weighted_f_score(predictions, diseases)
                f1_prior_based = weighted_f_score(prior_based_predicitons, diseases)
                print(f'heatmap_score={f1}, prior-based-score={f1_prior_based}, area_size={size}, function={func.__name__}, sim_threshold={sim_threshold}, encoding_categories={encoding_cats}')

heatmap_score=0.7199169060146015, prior-based-score=0.7198140693569967, area_size=2, function=amax, sim_threshold=0.6, encoding_categories=2
heatmap_score=0.7226297224517638, prior-based-score=0.7198140693569967, area_size=2, function=amin, sim_threshold=0.6, encoding_categories=2
heatmap_score=0.7226297224517638, prior-based-score=0.7198140693569967, area_size=2, function=mean, sim_threshold=0.6, encoding_categories=2
heatmap_score=0.7226297224517638, prior-based-score=0.7198140693569967, area_size=2, function=median, sim_threshold=0.6, encoding_categories=2
heatmap_score=0.7224276600063999, prior-based-score=0.7198140693569967, area_size=5, function=amax, sim_threshold=0.6, encoding_categories=2
heatmap_score=0.7226297224517638, prior-based-score=0.7198140693569967, area_size=5, function=amin, sim_threshold=0.6, encoding_categories=2
heatmap_score=0.7203412149261399, prior-based-score=0.7198140693569967, area_size=5, function=mean, sim_threshold=0.6, encoding_categories=2
heatmap_sco